# Joining University Data and GeoID + County IDs using City States from Patents

### Progress: Join created, a few values did not join and need to be fixed manually

## Pulling University Data and Patents Data From Box

In [86]:
from boxsdk import DevelopmentClient
import pandas as pd
import dotenv
import os
from io import BytesIO

In [87]:
# env_location =  'C:\\Users\\trent\\Documents\\Capstone' ##change as necessaru
os.chdir(env_location)
dotenv.load_dotenv() # register the .env file where passwords are stored

True

In [53]:
CLIENT_ID = os.getenv("box_client_id")
CLIENT_SECRET = os.getenv("box_client_secret")
# TOKEN = '' ##copy and paste from dev

In [54]:
from boxsdk import OAuth2, Client

oauth = OAuth2(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=TOKEN,
)

box = Client(oauth)

me = box.user().get()

print('logged into Box as', me.login)

logged into Box as txc5ve@virginia.edu


In [55]:
MY_FOLDER_ID = 0
my_folder = box.folder(MY_FOLDER_ID).get()
my_folder
items = my_folder.get_items()
items
for i in items:
    print(i)

<Box Folder - 166970315293 (Patents22)>


In [7]:
universities_box_id = '988549931799'
universities_file = box.file(universities_box_id).get()

In [8]:
universities_content = box.file(universities_box_id).content()

In [9]:
universities = pd.read_csv(BytesIO(universities_content))

In [33]:
## crosswalk not used for join
crosswalk_box_id = '987427719169'
crosswalk_file = box.file(crosswalk_box_id).get()
crosswalk_file
crosswalk_content = box.file(crosswalk_box_id).content()
crosswalk = pd.read_csv(BytesIO(crosswalk_content))

In [57]:
patents_box_id = '987961636758'
patents_file = box.file(patents_box_id).get()
patents_file
patents_content = box.file(patents_box_id).content()
patents = pd.read_csv(BytesIO(patents_content))

In [77]:
universities

,Unnamed: 0,Institution,Control,City,State,level
0,0,Arizona State University Campus Immersion,Public,tempe,AZ,r1
1,1,Auburn University,Public,auburn,AL,r1
2,2,Baylor University,Private (non-profit),waco,TX,r1
3,3,Binghamton University,Public,vestal,NY,r1
4,4,Boston College,Private (non-profit),chestnut hill,MA,r1
...,...,...,...,...,...,...
297,18,University of Texas Health Science Center at H...,Public,houston,TX,r3
298,19,University of Texas Health Science Center at S...,Public,san antonio,TX,r3
299,20,University of Texas Medical Branch,Public,galveston,TX,r3
300,21,University of Texas Southwestern Medical Center,Public,dallas,TX,r3


In [35]:
crosswalk.head(5)

,state,city,STATEFP,COUNTYFP,GEOID
0,AE,Apo,35.0,57.0,35057
1,AK,Alexander,40.0,51.0,00051
2,AK,Centerton,5.0,7.0,05007
3,AK,Fort Smith,5.0,131.0,05131
4,AK,Monticello,27.0,171.0,27171


## Cleaning Data for Join on City/State

In [45]:
universities.City = universities.City.str.lower()
universities.dtypes

Unnamed: 0      int64
Institution    object
Control        object
City           object
State          object
level          object
dtype: object

In [49]:
##crosswalk ultimately not used due to the typos in city names
crosswalk.city = crosswalk.city.str.lower()
crosswalk.dtypes
crosswalk.query("city == 'chestnut hill'")

,state,city,STATEFP,COUNTYFP,GEOID
1546,MA,chestnut hill,28.0,89.0,28089


In [58]:
patents.city = patents.city.str.lower()
patents

,Unnamed: 0,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county_name,geoID,ipc_section,team_size,inventors,men_inventors,women_inventors
0,0,7472557,CARRIER CORPORATION,2009,2004,11025351,US,farmington,CT,SAN JUAN,35045,F,4,3,3,0
1,1,7472744,CARRIER CORPORATION,2009,2005,11794273,US,farmington,CT,SAN JUAN,35045,F,2,1,1,0
2,2,7475757,OTIS ELEVATOR COMPANY,2009,2003,10554131,US,farmington,CT,SAN JUAN,35045,B,2,2,1,1
3,3,7479861,OTIS ELEVATOR COMPANY,2009,2003,10552383,US,farmington,CT,SAN JUAN,35045,H,10,10,8,2
4,4,7481069,CARRIER CORPORATION,2009,2005,11191679,US,farmington,CT,SAN JUAN,35045,F,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498686,1498686,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,webster city,IA,HAMILTON,19079,NaN,1,1,1,0
1498687,1498687,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,crawford,GA,OGLETHORPE,13221,NaN,4,4,3,1
1498688,1498688,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,minneapolis,KS,OTTAWA,20143,NaN,2,2,2,0
1498689,1498689,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,faulkton,SD,FAULK,46049,NaN,1,1,1,0


### Grouping PatenTs to get unique city, states, counties, and geoID

In [68]:
grouped = patents.groupby(['geoID','city', 'state', 'county_name'], as_index = False).count()
grouped

,geoID,city,state,county_name,Unnamed: 0,patent_number,assignee,grant_year,application_year,application_number,country,ipc_section,team_size,inventors,men_inventors,women_inventors
0,1003,bay minette,AL,BALDWIN,19,19,19,19,19,19,19,4,19,19,19,19
1,1003,fairhope,AL,BALDWIN,14,14,14,14,14,14,14,13,14,14,14,14
2,1003,foley,AL,BALDWIN,2,2,2,2,2,2,2,2,2,2,2,2
3,1003,gulf shores,AL,BALDWIN,3,3,3,3,3,3,3,2,3,3,3,3
4,1003,loxley,AL,BALDWIN,40,40,40,40,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10804,56041,lonetree,CO,UINTA,1,1,1,1,1,1,1,1,1,1,1,1
10805,56041,lyman,WY,UINTA,1,1,1,1,1,1,1,1,1,1,1,1
10806,56041,melburne,FL,UINTA,1,1,1,1,1,1,1,1,1,1,1,1
10807,56043,worland,WY,WASHAKIE,2,2,2,2,2,2,2,2,2,2,2,2


### Joining on City and State

In [73]:
university_geo_join = pd.merge(universities, grouped, left_on=['City', 'State'], right_on=['city','state'], how='left')

In [75]:
university_geo = university_geo_join[['Institution', 'Control', 'City', 'State', 'level', 'geoID', 'city', 'state', 'county_name']]

In [80]:
university_geo

,Institution,Control,City,State,level,geoID,city,state,county_name
0,Arizona State University Campus Immersion,Public,tempe,AZ,r1,4013.0,tempe,AZ,MARICOPA
1,Auburn University,Public,auburn,AL,r1,1081.0,auburn,AL,LEE
2,Baylor University,Private (non-profit),waco,TX,r1,48309.0,waco,TX,MCLENNAN
3,Binghamton University,Public,vestal,NY,r1,36007.0,vestal,NY,BROOME
4,Boston College,Private (non-profit),chestnut hill,MA,r1,28089.0,chestnut hill,MA,MADISON
...,...,...,...,...,...,...,...,...,...
297,University of Texas Health Science Center at H...,Public,houston,TX,r3,48201.0,houston,TX,HARRIS
298,University of Texas Health Science Center at S...,Public,san antonio,TX,r3,48029.0,san antonio,TX,BEXAR
299,University of Texas Medical Branch,Public,galveston,TX,r3,48167.0,galveston,TX,GALVESTON
300,University of Texas Southwestern Medical Center,Public,dallas,TX,r3,48113.0,dallas,TX,DALLAS


### Some Values did not Join

In [85]:
university_geo[university_geo['geoID'].isnull()]

,Institution,Control,City,State,level,geoID,city,state,county_name
22,George Mason University,Public,fairfax,VA,r1,NaN,NaN,NaN,NaN
31,Johns Hopkins University,Private (non-profit),baltimore,MD,r1,NaN,NaN,NaN,NaN
48,Old Dominion University,Public,norfolk,VA,r1,NaN,NaN,NaN,NaN
99,"University of Maryland, Baltimore County",Public,baltimore,MD,r1,NaN,NaN,NaN,NaN
133,University of Virginia,Public,charlottesville,VA,r1,NaN,NaN,NaN,NaN
139,Virginia Commonwealth University,Public,richmond,VA,r1,NaN,NaN,NaN,NaN
142,Washington University in St. Louis,Private (non-profit),saint louis,MO,r1,NaN,NaN,NaN,NaN
146,Air Force Institute of Technology Graduate Sch...,Public,wright-patterson afb,OH,r2,NaN,NaN,NaN,NaN
162,Central Michigan University,Public,mount pleasant,MI,r2,NaN,NaN,NaN,NaN
169,College of William and Mary,Public,williamsburg,VA,r2,NaN,NaN,NaN,NaN
